In [1]:
from faker import Faker
import numpy as np 
import pandas as pd

# importación de librerías propias
import utils.folders_tb as fold 
import utils.tools as tools
import utils.sql_tb as sql
from utils.mysql_driver import MySQL
import utils.mysql_driver as drv

fake = Faker('es_ES')
df_locales = fold.read_file_toDF('locales_REFINADO',';')
df_locales.drop(columns=['Unnamed: 0','Unnamed: 20'], inplace=True)

# recorrido del df de locales para generar los datos que sean fake
for i in range(len(df_locales)):
    if df_locales['DIRECCION'].isnull()[i] == True:
        # Generación de los campos de la dirección
        address_seed = fake.address()
        address_seed_list = list(address_seed.split())
        address_CP = address_seed_list[-1]
        address_city = address_seed_list[-2].replace(',', '')
        address_clean = address_seed.replace(address_CP, '')
        address_clean = address_clean.replace(address_city, '')
        address_clean = address_clean.replace('\n', '')
        address_clean = address_clean.replace(',', '')
        df_locales['DIRECCION'][i] = address_clean

    # generación de los datos de contacto de la empresa
    contacto = fake.first_name() + ' ' + fake.last_name() + ' ' + fake.last_name()
    df_locales['CONTACTO_EMPRESA'][i] = contacto # CONTACTO_EMPRESA	
    df_locales['EMAIL'][i] = fake.company_email() # EMAIL	
    df_locales['TELEFONO'][i] = fake.phone_number() # TELEFONO	

    # generación de los datos de accesibilidad
    # generación de la certificación (1 de cada 10 - garantiza la accesibilidad 0)
    if np.random.randint(0,101) > 90:
        df_locales['FAKE_ACCESS'][i] = 0
        df_locales['CERTIFICACION'][i] = 'Y'
    else:
        df_locales['CERTIFICACION'][i] = 'N'
        df_locales['FAKE_ACCESS'][i] = np.random.randint(0,2) + np.random.randint(0,2) # 0/1/2 alta/media/baja accesibilidad

    if df_locales['FAKE_ACCESS'][i] == 0:
        if df_locales['CERTIFICACION'][i] == 'Y':
            val = np.random.randint(1,10) + np.random.randint(0,5) + np.random.randint(0,20) + np.random.randint(0,40)
        else:
            val = np.random.randint(1,10) + np.random.randint(0,5) + np.random.randint(0,20)
        df_locales['VALORACIONES'][i] = val
        df_locales['ACUM_VERDE'][i] = round(val*0.8)
        df_locales['ACUM_AMARILLO'][i] = round(val*0.18)
        df_locales['ACUM_ROJO'][i] = round(val*0.02)
    elif df_locales['FAKE_ACCESS'][i] == 1:
        val = np.random.randint(1,5) + np.random.randint(0,5) + np.random.randint(0,10)
        df_locales['VALORACIONES'][i] = val
        df_locales['ACUM_VERDE'][i] = round(val*0.15)
        df_locales['ACUM_AMARILLO'][i] = round(val*0.7)
        df_locales['ACUM_ROJO'][i] = round(val*0.15)
    elif df_locales['FAKE_ACCESS'][i] == 2:
        val = np.random.randint(1,10) + np.random.randint(0,5) + np.random.randint(0,20)
        df_locales['VALORACIONES'][i] = val
        df_locales['ACUM_ROJO'][i] = round(val*0.8)
        df_locales['ACUM_AMARILLO'][i] = round(val*0.18)
        df_locales['ACUM_VERDE'][i] = round(val*0.02)

df_locales

ModuleNotFoundError: No module named 'utils'

In [62]:
fold.save_df_to_csv(df_locales,'locales')
sql.insert_sqlList (df_locales,'locales')

Connected to MySQL server [TurismoAccesible]
Executed 

DROP TABLE IF EXISTS locales

 successfully
Executed 

CREATE TABLE locales(
        ID INT(11) NOT NULL AUTO_INCREMENT,
        MOMENTO TIMESTAMP NOT NULL,
        CA VARCHAR(255) NOT NULL,PROVINCIA VARCHAR(255) NOT NULL,LATITUD VARCHAR(255) NOT NULL,LONGITUD VARCHAR(255) NOT NULL,NOMBRE_LOCAL VARCHAR(255) NOT NULL,DIRECCION VARCHAR(255) NOT NULL,CIUDAD VARCHAR(255) NOT NULL,TIPO VARCHAR(255) NOT NULL,SECTOR VARCHAR(255) NOT NULL,CONTACTO_EMPRESA VARCHAR(255) NOT NULL,EMAIL VARCHAR(255) NOT NULL,TELEFONO VARCHAR(255) NOT NULL,FAKE_ACCESS FLOAT(24) NOT NULL,ACUM_VERDE FLOAT(24) NOT NULL,ACUM_AMARILLO FLOAT(24) NOT NULL,ACUM_ROJO FLOAT(24) NOT NULL,VALORACIONES FLOAT(24) NOT NULL,CERTIFICACION VARCHAR(255) NOT NULL,IMAGEN VARCHAR(255) NOT NULL,
        PRIMARY KEY (ID))

 successfully
Executed 

INSERT INTO locales 
        (MOMENTO,CA,PROVINCIA,LATITUD,LONGITUD,NOMBRE_LOCAL,DIRECCION,CIUDAD,TIPO,SECTOR,CONTACTO_EMPRESA,EMAIL,TELEF

In [63]:
df_locales['VALORACIONES'].sum()

5282.0